In [5]:
# Wide display
from IPython.core.display import display, HTML
display(HTML("<style>#notebook-container { margin-left:-14px; width:calc(100% + 27px) !important; }</style>"))

In [6]:
import csv, datetime, fcntl, glob, json, math, os, re, subprocess, sys, time, urllib2, xml.dom.minidom, pytz
from dateutil import rrule
from dateutil import tz
import dateutil.parser
import numpy as np

# To install dateutil on Ubuntu
# sudo pip install python-dateutil

def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()

exec_ipynb('python-utils/esdr-library.ipynb')

In [7]:
esdr = Esdr(None)
product = esdr.get_product_by_name('PurpleAir')
product

{u'created': u'2017-11-20T15:17:13.000Z',
 u'creatorUserId': 3,
 u'defaultChannelSpecs': {},
 u'description': u'PurpleAir',
 u'id': 69,
 u'modified': u'2017-11-20T15:17:13.000Z',
 u'name': u'PurpleAir',
 u'prettyName': u'PurpleAir',
 u'vendor': u'PurpleAir'}

In [8]:
feeds = []
# Approx bounding box for Allegheny County
region = 'latitude>40.192204,latitude<40.674084,longitude<-79.688618,longitude>-80.361022'
offset = 0
while True:
    response = esdr.api('GET', '/api/v1/feeds', {'whereAnd' : 'productId=%d,%s' % (product['id'], region), 'offset': str(offset)})
    data = response['data']
    totalCount = data['totalCount']
    nrecs = len(data['rows'])
    feeds.extend(data['rows'])
    limit = data['limit']
    total_recvd = offset + nrecs
    #print(offset, nrecs, limit, totalCount)
    if total_recvd >= totalCount:
        break
    offset += nrecs

print 'Retrieved %d feeds' % len(feeds)
print 'First feed: %s' % feeds[0]

Retrieved 183 feeds
First feed: {u'apiKeyReadOnly': u'2bd4d40a3059b5ef56a3f6a6b78386569da2c2580fcd5c42e497072119f7b79c', u'minTimeSecs': 1522247982.349, u'name': u'Monaca,PA PurpleAir', u'created': u'2018-03-28T14:51:40.000Z', u'isPublic': 1, u'userId': 3, u'modified': u'2018-12-23T20:27:30.000Z', u'longitude': -80.311057, u'channelBounds': {u'channels': {u'PM2_5': {u'maxTimeSecs': 1545595853.279, u'minTimeSecs': 1522247982.349, u'maxValue': 3810.59, u'minValue': 0}, u'stats_v': {u'maxTimeSecs': 1545595853.279, u'minTimeSecs': 1522247982.349, u'maxValue': 970.64, u'minValue': 0}, u'stats_pm': {u'maxTimeSecs': 1545595853.279, u'minTimeSecs': 1522247982.349, u'maxValue': 970.64, u'minValue': 0}, u'temp_f': {u'maxTimeSecs': 1545595853.279, u'minTimeSecs': 1522247982.349, u'maxValue': 105, u'minValue': 27}, u'humidity': {u'maxTimeSecs': 1545595853.279, u'minTimeSecs': 1522247982.349, u'maxValue': 84, u'minValue': 11}, u'pressure': {u'maxTimeSecs': 1545595853.279, u'minTimeSecs': 1522247982

In [17]:
def sample_feed(feed, interval, from_epoch, to_epoch):
    feed_id = feed['id']
    export_url = 'https://esdr.cmucreatelab.org/api/v1/feeds/%d/channels/PM2.5_thinkspeak/export?from=%d&to=%d&format=json' % (feed_id, from_epoch, to_epoch)
    data = json.load(urllib2.urlopen(export_url))['data']
    
    
    print('Feed %d has %d samples' % (feed_id, len(data)))
    if len(data) == 0:
#         print('No samples, skipping')
        return None

    
    dat = []
    start_time = from_epoch
    end_time = from_epoch + interval - 1

    prevSample = 0
    while((end_time+1) < to_epoch):
        curbin = []
        for sample in data:
            if(sample[0] >= start_time and sample[0] <= end_time):
                curbin.append(sample[1])
        newSample = prevSample
        if(len(curbin) > 0):    
            newSample =  sum(curbin)/len(curbin)
        if(newSample == 0):
            newSample = prevSample
     
        start_time += interval
        end_time += (interval-1)
        
        dat.append(newSample)
        if(newSample != 0):
            prevSample = newSample
    
    
        
    row = ['feed_%d' % feed_id, feed['latitude'], feed['longitude']] + list(dat)
 
    return row

    
    

In [18]:
sensor =  sample_feed(feeds[110], 1800, 1532822400, 1532822400+86400)
sensor.pop(0)
print sensor
print len(np.array(sensor).astype(np.float32).tobytes())

Feed 14321 has 0 samples


AttributeError: 'NoneType' object has no attribute 'pop'

In [19]:
def output_raw_binary(cur_date):
    
    #Gets datetime from string
    date_datetime = datetime.datetime.strptime(cur_date, "%Y-%m-%d")
    #Convert to datetime
    epoch_time = (pytz.timezone('America/New_York').localize(date_datetime) - datetime.datetime(1970, 1, 1, tzinfo=pytz.utc)).total_seconds()
    raw_data = []
    bin_data = []
    count = 0
    for feed in feeds:

        #Adds 24 hours to starting epoch_time
        #Get each sensor feed sampled at 30 mins
        sensor = sample_feed(feed, 300, epoch_time, epoch_time+86400)
        
        #Adds all non-NULL sensors to raw/bin data
        if(sensor is not None and len(sensor) > 0):
            count += 1
            raw_data.append(sensor)
            sensor.pop(0)
            bin_data.append(sensor)
    
    #Convert to binary
    raw_binary = np.array(bin_data).astype(np.float32).tobytes()

    #Outputs binary file
    path = "aggregates/"
    bin_outfile = open(path + 'purpleair25_'  + cur_date + 'tmp.bin', 'w+')
    bin_outfile.write(raw_binary)
    bin_outfile.close()

    #Outputs raw json file
    path = "aggregates/"
    with open(path + 'purpleair25_'  + cur_date + 'tmp.json', 'w+') as outfile:  
        json.dump(raw_data, outfile)

    os.rename(path + 'purpleair25_'  + cur_date + 'tmp.json', path + 'purpleair25_'  + cur_date + '.json')
    os.rename(path + 'purpleair25_'  + cur_date + 'tmp.bin',  path + 'purpleair25_'  + cur_date + '.bin')


In [20]:
#Get current time
curDate = datetime.datetime.now().strftime("%Y-%m-%d")


curDate = '2018-10-01'
while(curDate != '2019-01-30'):
    print(curDate)
    output_raw_binary(curDate)
    curDate = (datetime.datetime.strptime(curDate, '%Y-%m-%d')+datetime.timedelta(days=1)).strftime("%Y-%m-%d")


2018-10-01
Feed 17199 has 0 samples
Feed 17200 has 0 samples
Feed 25600 has 0 samples
Feed 15876 has 1065 samples
Feed 15877 has 1065 samples
Feed 25601 has 0 samples
Feed 14945 has 1072 samples
Feed 14946 has 1071 samples
Feed 20581 has 1068 samples
Feed 20582 has 1073 samples
Feed 19958 has 0 samples
Feed 19959 has 0 samples
Feed 25765 has 1039 samples
Feed 25766 has 1031 samples
Feed 17964 has 1072 samples
Feed 17965 has 1068 samples
Feed 19703 has 1067 samples
Feed 19704 has 1066 samples
Feed 18986 has 1072 samples
Feed 18987 has 1071 samples
Feed 19028 has 1066 samples
Feed 19029 has 1070 samples
Feed 14927 has 1076 samples
Feed 14928 has 1068 samples
Feed 19956 has 1072 samples
Feed 19957 has 1071 samples
Feed 19302 has 1072 samples
Feed 19303 has 1072 samples
Feed 18679 has 1073 samples
Feed 18680 has 1072 samples
Feed 18106 has 1068 samples
Feed 18107 has 1076 samples
Feed 22641 has 0 samples
Feed 22642 has 0 samples
Feed 25697 has 0 samples
Feed 25698 has 0 samples
Feed 18681 

Feed 17984 has 0 samples
Feed 18096 has 1042 samples
Feed 18097 has 1039 samples
Feed 17785 has 0 samples
Feed 17786 has 0 samples
Feed 18271 has 1040 samples
Feed 18272 has 1038 samples
Feed 18269 has 0 samples
Feed 18270 has 0 samples
Feed 18210 has 1046 samples
Feed 18211 has 1044 samples
Feed 22300 has 0 samples
Feed 22301 has 0 samples
Feed 12698 has 1037 samples
Feed 12699 has 1039 samples
Feed 19988 has 0 samples
Feed 19989 has 0 samples
Feed 15431 has 0 samples
Feed 15432 has 0 samples
Feed 18443 has 0 samples
Feed 18444 has 0 samples
Feed 20451 has 1043 samples
Feed 20452 has 1043 samples
Feed 18437 has 0 samples
Feed 18438 has 0 samples
Feed 14551 has 1029 samples
Feed 14552 has 1043 samples
Feed 19984 has 0 samples
Feed 19985 has 0 samples
Feed 24258 has 0 samples
Feed 24259 has 0 samples
Feed 18104 has 1041 samples
Feed 18105 has 1040 samples
Feed 19850 has 1050 samples
Feed 19851 has 1043 samples
Feed 19154 has 1044 samples
Feed 19155 has 1049 samples
Feed 20351 has 0 samp

Feed 12940 has 0 samples
Feed 12941 has 0 samples
Feed 12952 has 0 samples
Feed 12953 has 0 samples
Feed 12948 has 0 samples
Feed 12949 has 0 samples
Feed 12950 has 0 samples
Feed 12951 has 0 samples
Feed 12942 has 0 samples
Feed 12943 has 0 samples
Feed 12946 has 0 samples
Feed 12947 has 0 samples
Feed 12958 has 0 samples
Feed 12959 has 0 samples
Feed 12954 has 0 samples
Feed 12955 has 0 samples
Feed 17888 has 0 samples
Feed 17889 has 0 samples
Feed 24309 has 0 samples
Feed 24310 has 0 samples
Feed 17783 has 1067 samples
Feed 17784 has 1065 samples
Feed 17195 has 1071 samples
Feed 17196 has 1071 samples
Feed 18827 has 0 samples
Feed 18828 has 0 samples
Feed 14314 has 0 samples
Feed 14315 has 0 samples
Feed 14316 has 0 samples
Feed 14317 has 0 samples
Feed 14318 has 0 samples
Feed 14319 has 0 samples
Feed 14320 has 0 samples
Feed 14321 has 0 samples
Feed 14322 has 1067 samples
Feed 14323 has 1063 samples
Feed 25790 has 0 samples
Feed 19846 has 0 samples
Feed 19847 has 0 samples
Feed 17

Feed 19956 has 1078 samples
Feed 19957 has 1075 samples
Feed 19302 has 1077 samples
Feed 19303 has 1079 samples
Feed 18679 has 1075 samples
Feed 18680 has 1072 samples
Feed 18106 has 1077 samples
Feed 18107 has 1079 samples
Feed 22641 has 0 samples
Feed 22642 has 0 samples
Feed 25697 has 0 samples
Feed 25698 has 0 samples
Feed 18681 has 1076 samples
Feed 18682 has 1074 samples
Feed 21992 has 1069 samples
Feed 21993 has 1077 samples
Feed 21988 has 1070 samples
Feed 21989 has 1068 samples
Feed 21990 has 1067 samples
Feed 21991 has 1073 samples
Feed 21994 has 1072 samples
Feed 21995 has 1068 samples
Feed 21986 has 0 samples
Feed 21987 has 0 samples
Feed 19707 has 1073 samples
Feed 19708 has 1072 samples
Feed 14761 has 0 samples
Feed 14762 has 0 samples
Feed 14763 has 0 samples
Feed 14764 has 0 samples
Feed 14768 has 1051 samples
Feed 14769 has 1074 samples
Feed 16691 has 0 samples
Feed 16692 has 0 samples
Feed 16693 has 0 samples
Feed 16694 has 0 samples
Feed 16695 has 0 samples
Feed 1669

Feed 14552 has 894 samples
Feed 19984 has 0 samples
Feed 19985 has 0 samples
Feed 24258 has 0 samples
Feed 24259 has 0 samples
Feed 18104 has 899 samples
Feed 18105 has 898 samples
Feed 19850 has 895 samples
Feed 19851 has 893 samples
Feed 19154 has 898 samples
Feed 19155 has 895 samples
Feed 20351 has 0 samples
Feed 20352 has 0 samples
Feed 21493 has 896 samples
Feed 21494 has 896 samples
Feed 21497 has 898 samples
Feed 21498 has 897 samples
Feed 13776 has 0 samples
Feed 13777 has 0 samples
Feed 17803 has 0 samples
Feed 17804 has 0 samples
Feed 17807 has 526 samples
Feed 17808 has 897 samples
Feed 16771 has 897 samples
Feed 16772 has 898 samples
2018-10-08
Feed 17199 has 0 samples
Feed 17200 has 0 samples
Feed 25600 has 0 samples
Feed 15876 has 0 samples
Feed 15877 has 0 samples
Feed 25601 has 0 samples
Feed 14945 has 0 samples
Feed 14946 has 0 samples
Feed 20581 has 0 samples
Feed 20582 has 0 samples
Feed 19958 has 0 samples
Feed 19959 has 0 samples
Feed 25765 has 0 samples
Feed 2576

...

Feed 12699 has 725 samples
Feed 19988 has 1026 samples
Feed 19989 has 726 samples
Feed 15431 has 880 samples
Feed 15432 has 725 samples
Feed 18443 has 0 samples
Feed 18444 has 0 samples
Feed 20451 has 0 samples
Feed 20452 has 0 samples
Feed 18437 has 0 samples
Feed 18438 has 0 samples
Feed 14551 has 906 samples
Feed 14552 has 904 samples
Feed 19984 has 0 samples
Feed 19985 has 0 samples
Feed 24258 has 0 samples
Feed 24259 has 0 samples
Feed 18104 has 666 samples
Feed 18105 has 666 samples
Feed 19850 has 863 samples
Feed 19851 has 879 samples
Feed 19154 has 1061 samples
Feed 19155 has 1061 samples
Feed 20351 has 0 samples
Feed 20352 has 0 samples
Feed 21493 has 667 samples
Feed 21494 has 667 samples
Feed 21497 has 697 samples
Feed 21498 has 680 samples
Feed 13776 has 0 samples
Feed 13777 has 0 samples
Feed 17803 has 0 samples
Feed 17804 has 0 samples
Feed 17807 has 680 samples
Feed 17808 has 695 samples
Feed 16771 has 697 samples
Feed 16772 has 698 samples
2019-01-22
Feed 17199 has 0 sa

Feed 17888 has 1078 samples
Feed 17889 has 1077 samples
Feed 24309 has 0 samples
Feed 24310 has 0 samples
Feed 17783 has 1079 samples
Feed 17784 has 1077 samples
Feed 17195 has 912 samples
Feed 17196 has 1039 samples
Feed 18827 has 343 samples
Feed 18828 has 807 samples
Feed 14314 has 0 samples
Feed 14315 has 0 samples
Feed 14316 has 0 samples
Feed 14317 has 0 samples
Feed 14318 has 0 samples
Feed 14319 has 0 samples
Feed 14320 has 0 samples
Feed 14321 has 0 samples
Feed 14322 has 1080 samples
Feed 14323 has 1078 samples
Feed 25790 has 1080 samples
Feed 19846 has 0 samples
Feed 19847 has 0 samples
Feed 17892 has 1080 samples
Feed 17893 has 1080 samples
Feed 13902 has 0 samples
Feed 13903 has 0 samples
Feed 17978 has 1080 samples
Feed 17979 has 1080 samples
Feed 18412 has 1076 samples
Feed 18413 has 1072 samples
Feed 17966 has 213 samples
Feed 17967 has 0 samples
Feed 25789 has 1080 samples
Feed 17970 has 1080 samples
Feed 17971 has 1080 samples
Feed 18153 has 1070 samples
Feed 18154 ha

Feed 21989 has 0 samples
Feed 21990 has 0 samples
Feed 21991 has 0 samples
Feed 21994 has 0 samples
Feed 21995 has 0 samples
Feed 21986 has 0 samples
Feed 21987 has 0 samples
Feed 19707 has 585 samples
Feed 19708 has 1015 samples
Feed 14761 has 0 samples
Feed 14762 has 0 samples
Feed 14763 has 0 samples
Feed 14764 has 0 samples
Feed 14768 has 0 samples
Feed 14769 has 0 samples
Feed 16691 has 0 samples
Feed 16692 has 0 samples
Feed 16693 has 0 samples
Feed 16694 has 0 samples
Feed 16695 has 0 samples
Feed 16696 has 0 samples
Feed 16697 has 0 samples
Feed 15499 has 0 samples
Feed 15500 has 0 samples
Feed 18084 has 1072 samples
Feed 18085 has 1069 samples
Feed 19895 has 0 samples
Feed 19896 has 0 samples
Feed 19741 has 0 samples
Feed 19742 has 0 samples
Feed 12956 has 0 samples
Feed 12957 has 0 samples
Feed 18702 has 833 samples
Feed 18703 has 999 samples
Feed 12944 has 0 samples
Feed 12945 has 0 samples
Feed 12940 has 0 samples
Feed 12941 has 0 samples
Feed 12952 has 0 samples
Feed 12953

Feed 21494 has 1080 samples
Feed 21497 has 1071 samples
Feed 21498 has 1078 samples
Feed 13776 has 0 samples
Feed 13777 has 0 samples
Feed 17803 has 0 samples
Feed 17804 has 0 samples
Feed 17807 has 1078 samples
Feed 17808 has 1076 samples
Feed 16771 has 1078 samples
Feed 16772 has 1078 samples
2019-01-27
Feed 17199 has 0 samples
Feed 17200 has 0 samples
Feed 25600 has 0 samples
Feed 15876 has 1037 samples
Feed 15877 has 1029 samples
Feed 25601 has 1026 samples
Feed 14945 has 750 samples
Feed 14946 has 901 samples
Feed 20581 has 0 samples
Feed 20582 has 0 samples
Feed 19958 has 0 samples
Feed 19959 has 0 samples
Feed 25765 has 1079 samples
Feed 25766 has 1077 samples
Feed 17964 has 1078 samples
Feed 17965 has 1077 samples
Feed 19703 has 212 samples
Feed 19704 has 767 samples
Feed 18986 has 1077 samples
Feed 18987 has 1076 samples
Feed 19028 has 1080 samples
Feed 19029 has 1080 samples
Feed 14927 has 915 samples
Feed 14928 has 967 samples
Feed 19956 has 1080 samples
Feed 19957 has 1080 

Feed 17979 has 855 samples
Feed 18412 has 854 samples
Feed 18413 has 851 samples
Feed 17966 has 0 samples
Feed 17967 has 0 samples
Feed 25789 has 851 samples
Feed 17970 has 855 samples
Feed 17971 has 854 samples
Feed 18153 has 855 samples
Feed 18154 has 855 samples
Feed 17983 has 0 samples
Feed 17984 has 0 samples
Feed 18096 has 853 samples
Feed 18097 has 852 samples
Feed 17785 has 855 samples
Feed 17786 has 855 samples
Feed 18271 has 0 samples
Feed 18272 has 0 samples
Feed 18269 has 0 samples
Feed 18270 has 773 samples
Feed 18210 has 518 samples
Feed 18211 has 0 samples
Feed 22300 has 855 samples
Feed 22301 has 855 samples
Feed 12698 has 851 samples
Feed 12699 has 847 samples
Feed 19988 has 804 samples
Feed 19989 has 849 samples
Feed 15431 has 846 samples
Feed 15432 has 853 samples
Feed 18443 has 0 samples
Feed 18444 has 0 samples
Feed 20451 has 0 samples
Feed 20452 has 0 samples
Feed 18437 has 0 samples
Feed 18438 has 0 samples
Feed 14551 has 841 samples
Feed 14552 has 841 samples
Fe